# 用Cython包装C++代码

Cython最大的作用其实是作为C++代码和python代码的桥梁,比如我们已经有一个C++写的程序了,但我们希望让python可以调用它,传统的做法是使用ctypes或者cffi作为桥,但这种方式需要有相当的C/C++知识.Cython的话基本可以无痛进行C++代码的包装,我们通过包装一个例子来看看cython是如何包装c++代码的,

## 最简单的一个例子

这个例子用来介绍Cython包装C/C++代码的步骤.例子是一个长方形类,C++代码部分如下:

In [15]:
%%writefile Rectangle.h

namespace shapes {
    class Rectangle {
    public:
        int x0, y0, x1, y1;
        Rectangle();
        Rectangle(int x0, int y0, int x1, int y1);
        ~Rectangle();
        int getArea();
        void getSize(int* width, int* height);
        void move(int dx, int dy);
    };
}

Overwriting Rectangle.h


In [16]:
%%writefile Rectangle.cpp
#include "Rectangle.h"

namespace shapes {

  Rectangle::Rectangle() { }

    Rectangle::Rectangle(int X0, int Y0, int X1, int Y1) {
        x0 = X0;
        y0 = Y0;
        x1 = X1;
        y1 = Y1;
    }

    Rectangle::~Rectangle() { }

    int Rectangle::getArea() {
        return (x1 - x0) * (y1 - y0);
    }

    void Rectangle::getSize(int *width, int *height) {
        (*width) = x1 - x0;
        (*height) = y1 - y0;
    }

    void Rectangle::move(int dx, int dy) {
        x0 += dx;
        y0 += dy;
        x1 += dx;
        y1 += dy;
    }

}

Overwriting Rectangle.cpp


## 用于包装的pyx文件

要包装C++文件,我们得先在cython中声明出这个C++的类,在cython中申明C或者C++的内容(接口)需要使用`cdef extern from ....`这种语法(外部声明).

在

In [23]:
%%writefile rect.pyd
#cython: language_level=3
# distutils: language = c++
# distutils: sources = Rectangle.cpp



Overwriting rect.pyd


In [25]:
%%writefile rect.pyx
#cython: language_level=3
# distutils: language = c++
# distutils: sources = Rectangle.cpp

cdef extern from "Rectangle.h" namespace "shapes":
    cdef cppclass Rectangle:
        Rectangle() except +
        Rectangle(int, int, int, int) except +
        int x0, y0, x1, y1
        int getArea()
        void getSize(int* width, int* height)
        void move(int, int)

cdef class PyRectangle:
    cdef Rectangle c_rect      # hold a C++ instance which we're wrapping
    def __cinit__(self, int x0, int y0, int x1, int y1):
        self.c_rect = Rectangle(x0, y0, x1, y1)
    def get_area(self):
        return self.c_rect.getArea()
    def get_size(self):
        cdef int width, height
        self.c_rect.getSize(&width, &height)
        return width, height
    def move(self, dx, dy):
        self.c_rect.move(dx, dy)

Overwriting rect.pyx


这样，我们就完成了C++的封装。而且从Python的开发角度来看，这个扩展类型看起来和感觉就像一个本地定义的Rectangle类。
需要注意的是，如果我们需要额外的属性设置方法，可以自己再添加.

## setup.py的写法

我们的setup.py和之前差不多的写法

In [26]:
%%writefile setup.py

from distutils.core import setup
from Cython.Build import cythonize
 
setup(
    name = "rectangleapp",
    ext_modules = cythonize('*.pyx')
)

Overwriting setup.py


In [27]:
!python setup.py build_ext --inplace

Compiling rect.pyx because it changed.
[1/1] Cythonizing rect.pyx
running build_ext


Traceback (most recent call last):
  File "setup.py", line 7, in <module>
    ext_modules = cythonize('*.pyx')
  File "C:\Users\Administrator\Anaconda3\lib\distutils\core.py", line 148, in setup
    dist.run_commands()
  File "C:\Users\Administrator\Anaconda3\lib\distutils\dist.py", line 955, in run_commands
    self.run_command(cmd)
  File "C:\Users\Administrator\Anaconda3\lib\distutils\dist.py", line 974, in run_command
    cmd_obj.run()
  File "C:\Users\Administrator\Anaconda3\lib\distutils\command\build_ext.py", line 308, in run
    force=self.force)
  File "C:\Users\Administrator\Anaconda3\lib\distutils\ccompiler.py", line 1031, in new_compiler
    return klass(None, dry_run, force)
  File "C:\Users\Administrator\Anaconda3\lib\distutils\cygwinccompiler.py", line 282, in __init__
    CygwinCCompiler.__init__ (self, verbose, dry_run, force)
  File "C:\Users\Administrator\Anaconda3\lib\distutils\cygwinccompiler.py", line 157, in __init__
    self.dll_libraries = get_msvcr()
  File "C